In [38]:
# Import all required libraries
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

np.random.seed(0)
%matplotlib inline

X = np.genfromtxt('data/X_train.txt', delimiter=None) 
Y = np.genfromtxt('data/Y_train.txt', delimiter=None) 
X,Y = ml.shuffleData(X,Y)
Xtr, Xva, Ytr, Yva = ml.splitData(X,Y,0.75)
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

In [50]:
a= [1,2,3]
b = np.ones(len(a))
print(b)
print(np.vstack((b-a, a)).T)


[ 1.  1.  1.]
[[ 0.  1.]
 [-1.  2.]
 [-2.  3.]]


In [59]:
boosts = []
n_boosts = 300

Ytr_ = np.copy(Ytr)  # We're going to copy the data becuase each booster iteration we're going to mess with it.
#print(Ytr_.shape)
for i in range(n_boosts):
    tree = ml.dtree.treeRegress(Xtr, Ytr_, maxDepth=2)
    """you can substitute it with linear regression"""
    boosts.append(tree)
    # Now "learning" from out mistakes.
    Ytr_ -= tree.predict(Xtr)
    #print(Ytr_)

In [60]:
class Gradient_Boosted_Tree(ml.base.classifier):
    def __init__(self, boosts):
        """Constructs a Gradient_boosted_Tree class with a set of boosts. """
        self.boosts = boosts
    
    def predictSoft(self, X):
        """Predicts regression values using boosting. """
        n_boosts = len(self.boosts)
        preds = [self.boosts[i].predict(X) for i in range(n_boosts)]
        #print(preds)
        p1 = np.sum(preds, axis=0)
        p0 = np.ones(len(p1)) - p1
        pre = np.vstack((p0, p1)).T
        return pre


In [61]:
gbt = Gradient_Boosted_Tree(boosts)
gbt.classes = np.unique(Y)

probs = gbt.predictSoft(Xte)
print(probs)
print("{0:>15}: {1:.4f}".format('Train AUC', gbt.auc(Xtr, Ytr)))
print("{0:>15}: {1:.4f}".format('Validation AUC', gbt.auc(Xva, Yva)))

[[ 0.27425763  0.72574237]
 [ 0.55554064  0.44445936]
 [ 0.77187238  0.22812762]
 ..., 
 [ 0.86243778  0.13756222]
 [ 0.8823448   0.1176552 ]
 [ 0.93291025  0.06708975]]
      Train AUC: 0.7490
 Validation AUC: 0.7208


In [ ]:
Yte = np.vstack((np.arange(Xte.shape[0]), gbt.predictSoft(Xte)[:,1])).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('Y_submit2.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')